In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from random import choice
import pickle
import pandas as pd
from collections import defaultdict
import lyricsgenius
from essential_generators import DocumentGenerator
import re
import collections
from pymongo import MongoClient

In [2]:
def get_keys():
    '''
    Access API keys from external file and placed in a list. 

    Parameters
    ----------
    None:

    Returns
    ----------
    API_KEYS : (list)
        Return API keys used for connected to Spotify and Genius APIs.
    '''
    f = open('api_keys.txt', 'r')
    API_KEYS = f.readlines()
    
    for idx, key in enumerate(API_KEYS):
        API_KEYS[idx] = key.replace('\n', '')
        
    return API_KEYS

In [3]:
def get_album_ids_from_query(album_search):
    '''
    Pluck unique album IDs from API query, iterrates through paginated query, return list of album IDs from query.

    Parameters
    ----------
    album_query: (dict)
        Dictionary querried from Spotify API.

    Returns
    ----------
    album_ids : (list)
        Return album IDs from API query.
    '''
    album_info = album_search['albums']['items']
    album_ids = []
    
    for n in range(len(album_info)):
        album_ids.append(album_info[n]['id'])
        
        
    return album_ids

In [4]:
#random gen functions for api query
def get_rand_offset():
    return choice(range(10))
def get_rand_word(gen):
    return gen.word()

In [5]:
def album_builder_test(album):
    album_dict = defaultdict(list)
    
    album_dict['artist'].append(album['artists'][0]['name'])
    album_dict['artist_id'].append(album['artists'][0]['id'])
    album_dict['album'].append(album['name'])
    album_dict['album_type'].append(album['album_type'])
    album_dict['album_id'].append(album['id'])
    album_dict['album_label'].append(album['label'])
    if album['external_ids']:
        album_dict['upc_code'].append(album['external_ids']['upc'])
    else: 
        pass
#     album_dict['upc_code'].append(album['external_ids']['upc'])
    album_dict['album_popularity'].append(album['popularity'])
    album_dict['release_date'].append(album['release_date'])
    album_dict['release_prec'].append(album['release_date_precision'])
    album_dict['genres'].append(album['genres'])
    album_dict['album_release'].append(album['name'] + ' by ' + album['artists'][0]['name'])

    tracks, track_ids = [], [] 
    for track in album['tracks']['items']:
        tracks.append(track['name'])
        track_ids.append(track['id'])
    
    
    return album_dict, tracks, track_ids

In [6]:
def decode_lyrics(s):
    s = s.encode('ascii', 'ignore')
    s = s.decode()
    s = s.replace('\n', ' ')
    s = s.replace('-', ' ')
    s = re.sub(r'[\[].*?[\)\]]', ' ', s)
    
    return s

In [7]:
def lyrics_builder(song_names, artist, album_id, genius):
    '''
    songs: lst
    artist: str
    genius: api obj
    '''
    lyrics_dict = defaultdict(list)

    lyrics_dict['album_id'] = album_id
    album_lyrics = ''
    for song in song_names:
        song_search = genius.search_song(song, artist)
        
        if song_search == None:
            del song_search
            continue
        else:
            album_lyrics += song_search.lyrics;
            del song_search
    
    lyrics_dict['lyrics'] = decode_lyrics(album_lyrics)
    
    return lyrics_dict

In [8]:
def genres_builder(album_id, genres):
    genre_dict = defaultdict(list)
    
    genre_dict['album_id'] = album_id
    genre_dict['genres'] = genres
    
    return genre_dict

In [9]:
def audio_features_builder(album_id, audio_features):
    audio_feat_dict = defaultdict(list)
    keys_to_pop = ['type', 'id', 'uri', 'track_href', 'analysis_url']
    track_feats = ['danceability', 'energy', 'key', 
                    'loudness', 'mode', 'speechiness', 'acousticness',
                   'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
    
    
    audio_feat_dict['album_id'] = album_id
    for track_info in audio_features:
        if track_info == None:
            continue
        [track_info.pop(key) for key in keys_to_pop]
        
    counter = collections.Counter()
    
    for d in audio_features:
        counter.update(d)
        
    sum_ = dict(counter)
    for k, v in sum_.items():
        audio_feat_dict[k] = v / len(audio_features)

    return audio_feat_dict

# Test and build df

In [10]:
# CONST and API instantiation

client_id, client_secret, client_access_token = get_keys()
gen = DocumentGenerator()
q_type='album'
limit = 50
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id,
                                                          client_secret=client_secret))
genius = lyricsgenius.Genius(client_access_token=client_access_token)

## ---Test single album ---
### multi-single iterations

In [14]:
test_id = '3Wnh6MgbUQZaaMJh9ThSKz'
# album_tracks = sp.album_tracks(test_id)
album = sp.album(test_id)

In [15]:
album_dict, tracks, track_ids = album_builder_test(album)
#to mongo, to genius + album_dict['artist'][0], to get features

In [16]:
artist = album_dict['artist'][0]
artist_id = album_dict['artist_id'][0]
album_id = album_dict['album_id'][0]

In [17]:
genres = sp.artist(artist_id)['genres']

In [26]:
audio_features = sp.audio_features(track_ids)

In [19]:
genres_dict = genres_builder(album_id, genres)

In [20]:
lyrics_dict = lyrics_builder(tracks, artist, album_id, genius)
# to mongo

Searching for "The One" by Cloudy Nueve...
Done.
Searching for "Pain Pills" by Cloudy Nueve...
Done.
Searching for "Pocket Change" by Cloudy Nueve...
Done.
Searching for "Treason" by Cloudy Nueve...
Done.
Searching for "Stranger to Love" by Cloudy Nueve...
Done.
Searching for "Bandana" by Cloudy Nueve...
Done.
Searching for ".45 Sliding" by Cloudy Nueve...
Done.
Searching for "No Stress" by Cloudy Nueve...
Done.
Searching for "Up!" by Cloudy Nueve...
Done.
Searching for "Why Do You Lie" by Cloudy Nueve...
Done.
Searching for "On My Own" by Cloudy Nueve...
Done.
Searching for "Afterlife" by Cloudy Nueve...
Done.


In [12]:
# for track_info in audio_features:
# #     print(track_info)

In [1]:
features_dict = audio_features_builder(album_id, audio_features)

NameError: name 'audio_features_builder' is not defined

## --- Test complete ---

### Testing dictionaries for mongo db

In [25]:
album_df = pd.DataFrame(album_dict)
album_df

,artist,artist_id,album,album_type,album_id,album_label,upc_code,album_popularity,release_date,release_prec,genres,album_release
0,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,After Hours,album,4yP0hdKOZPNshxUOjY0cZj,Republic Records,00602508924224,96,2020-03-20,day,[],After Hours by The Weeknd


In [77]:
lyric_df = pd.DataFrame(lyrics_dict, index=[0])
lyric_df

,album_id,lyrics
0,4yP0hdKOZPNshxUOjY0cZj,Take off my disguise I'm living someone else...


In [27]:
genres_df = pd.DataFrame(genres_dict)
genres_df

,album_id,genres
0,4yP0hdKOZPNshxUOjY0cZj,canadian contemporary r&b
1,4yP0hdKOZPNshxUOjY0cZj,canadian pop
2,4yP0hdKOZPNshxUOjY0cZj,pop


In [28]:
feat_df = pd.DataFrame(features_dict, index=[0])
feat_df

,album_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,4yP0hdKOZPNshxUOjY0cZj,0.501286,0.6405,3.285714,-7.085929,0.428571,0.068607,0.122286,0.011832,0.193564,0.250564,121.552929,241255.285714,3.928571


# create list of album ids (+ old 511 new) shoot for 100k
## have 150k+

# DO NOT TEST -- TEST BELOW FROM SINGLE CELL

In [24]:
client = MongoClient('localhost', 27017)
db = client['album_info']
db['album_id_master']
db['albums']
db['lyrics']
db['genres']
db['feat_df']
master_coll = db['album_master_id']
album_coll = db['albums']
lyric_coll = db['lyrics']
genre_coll = db['genres']
feat_coll = db['feat_df']

In [25]:
'''dicts for mongo'''
album_coll.insert_one(album_dict)

In [26]:
lyric_coll.insert_one(lyrics_dict)

In [27]:
genre_coll.insert_one(genres_dict)

In [28]:
feat_coll.insert_one(features_dict)

# automate process

## test unique album pulls
# DO NOT RUN

In [29]:
plucked_album_ids = []
BASELINE_ID_LIST = []
dfs = pickle.load(open('dfs_w_lyrics.pkl', 'rb'))
df = pd.concat(dfs)
album_ids_baseline = df['album_id'].unique()

In [ ]:
for album_ids in album_ids_baseline:
    BASELINE_ID_LIST.append(album_ids)

In [30]:
for _ in range(10000):
    album_query = sp.search(q=get_rand_word(gen), limit=limit, offset=get_rand_offset(), type=q_type)
    plucked_album_ids.append(get_album_ids_from_query(album_query))
# flattened_album_ids = [id_ for sublist in plucked_album_ids for id_ in sublist]

In [31]:
flattened_album_ids = [id_ for sublist in plucked_album_ids for id_ in sublist]
unique_album_ids = pd.Series(flattened_album_ids).nunique()

In [32]:
print(f'{len(flattened_album_ids)} plucked, {unique_album_ids} are unique')

435215 plucked, 156285 are unique


In [33]:
for id_ in MASTER_IDS:
    if id_ not in set(flattened_album_ids):
        flattened_album_ids.append(id_)

In [34]:
unique_album_ids = pd.Series(flattened_album_ids).nunique()
print(f'{len(flattened_album_ids)} plucked, {unique_album_ids} are unique')

435215 plucked, 156285 are unique


In [38]:
unique_ids = pd.Series(flattened_album_ids).unique()

In [40]:
#pickle.dump(unique_ids, open('MASTER_ALBUM_IDS.pkl', 'wb'))

## test stop

# Process albums
## NOTE-TO-SELF: cell run number should on 11

In [13]:
unique_ids = pickle.load(open('MASTER_ALBUM_IDS.pkl', 'rb'))

In [14]:
# mongo instant
client = MongoClient('localhost', 27017)
db = client['album_info']
db['album_id_master']
db['albums']
db['lyrics']
db['genres']
db['feat_df']
master_coll = db['album_master_id']
album_coll = db['albums']
lyric_coll = db['lyrics']
genre_coll = db['genres']
feat_coll = db['feat_df']

In [15]:
from tqdm.notebook import tqdm

In [16]:
ids_sent_to_mongo = []

In [17]:
for id_ in tqdm(unique_ids):
    print(f'Pulling album id {id_}')
    album = sp.album(id_)
    album_dict, tracks, track_ids = album_builder_test(album)
    
    artist = album_dict['artist'][0]
    artist_id = album_dict['artist_id'][0]
    album_id = album_dict['album_id'][0]

    genres = sp.artist(artist_id)['genres']
    audio_features = sp.audio_features(track_ids)
    
    genres_dict = genres_builder(album_id, genres)
    lyrics_dict = lyrics_builder(tracks, artist, album_id, genius)
    features_dict = audio_features_builder(album_id, audio_features)
    
    '''dicts for mongo'''
    album_coll.insert_one(album_dict)
    lyric_coll.insert_one(lyrics_dict)
    genre_coll.insert_one(genres_dict)
    feat_coll.insert_one(features_dict)
    
    ids_sent_to_mongo.append(id_)

Pulling album id 3RoG4xBbhThunES2bojEm7
Searching for "Connect Me" by Eleanor Arroway...
No results found for: 'Connect Me Eleanor Arroway'
Searching for "Touch of My Soul" by Eleanor Arroway...
No results found for: 'Touch of My Soul Eleanor Arroway'
Searching for "Finding My Heart" by Eleanor Arroway...
No results found for: 'Finding My Heart Eleanor Arroway'
Searching for "Spiritual Entanglement" by Eleanor Arroway...
No results found for: 'Spiritual Entanglement Eleanor Arroway'
Pulling album id 1awdBA8DLv6G5eBvdGxk8T
Searching for "Where I Want To Be" by The Dangerous Summer...
Done.
Searching for "Settle Down" by The Dangerous Summer...
Done.
Searching for "Weathered" by The Dangerous Summer...
Done.
Searching for "Symmetry" by The Dangerous Summer...
Done.
Searching for "Surfaced" by The Dangerous Summer...
Done.
Searching for "A Space To Grow" by The Dangerous Summer...
Done.
Searching for "Reach For The Sun" by The Dangerous Summer...
Done.
Searching for "The Permanent Rain" b

Done.
Searching for "Stand Out" by Philthy Rich...
Done.
Searching for "No Questions (feat. Yella Beezy)" by Philthy Rich...
Done.
Searching for "7 Mile x Seminary" by Philthy Rich...
Done.
Searching for "Expensive Taste (feat. Lil Migo)" by Philthy Rich...
Done.
Searching for "Real Hate" by Philthy Rich...
Done.
Searching for "Out The Hood" by Philthy Rich...
Done.
Searching for "Heart Anemic (feat. Kollision & Ball Greezy)" by Philthy Rich...
Done.
Searching for "Rich Gannon (feat. BlueBucksClan)" by Philthy Rich...
Done.
Searching for "Want Me Dead (feat. Bankroll Freddie)" by Philthy Rich...
Done.
Searching for "Ratchet Ass Bitch (feat. 1TakeJay)" by Philthy Rich...
Done.
Searching for "Fake Rumors" by Philthy Rich...
Done.
Searching for "Stand For Something" by Philthy Rich...
Done.
Searching for "Pouring Rose" by Philthy Rich...
Done.
Searching for "No Love (feat. Jackboy)" by Philthy Rich...
Done.
Searching for "Big 6 x Big 59 (feat. Icewear Vezzo)" by Philthy Rich...
Done.
Sear

Done.
Searching for "Father" by Demi Lovato...
Done.
Pulling album id 3Qu13eNn5FYNuFPwgBG6dO
Searching for "Concept Nothing" by Night Lovell...
Done.
Searching for "Live Television" by Night Lovell...
Done.
Searching for "Beneath" by Night Lovell...
Done.
Searching for "Sometimes Not Serious" by Night Lovell...
Done.
Searching for "The Renegade Never Dies" by Night Lovell...
Done.
Searching for "Off Air" by Night Lovell...
Done.
Searching for "Forget About Me" by Night Lovell...
Done.
Searching for "Dead Lovell" by Night Lovell...
Done.
Searching for "James Cameron Took My Bitch (Interlude)" by Night Lovell...
Done.
Searching for "Light from the Car" by Night Lovell...
Done.
Searching for "Material Female" by Night Lovell...
Done.
Searching for "Trees of the Valley" by Night Lovell...
Done.
Searching for "Questioning Your Theory" by Night Lovell...
Done.
Searching for "Deira City Centre" by Night Lovell...
Done.
Searching for "Dark Light" by Night Lovell...
Done.
Pulling album id 1h3x5

Done.
Searching for "Pink Toes (Offset, DaBaby feat. Gunna)" by Quality Control...
Done.
Searching for "Dead Man Walking" by Quality Control...
Done.
Searching for "100 Racks (Offset feat. Playboi Carti)" by Quality Control...
Done.
Searching for "Double Trouble (Quavo feat. Meek Mill)" by Quality Control...
Done.
Searching for "Bless Em (Takeoff feat. Travis Scott)" by Quality Control...
Done.
Searching for "Like That (City Girls, Stefflon Don feat. Renni Rucci, Mustard)" by Quality Control...
Done.
Searching for "Big Rocks (Offset feat. Young Thug)" by Quality Control...
Done.
Searching for "Virgil" by Quality Control...
Done.
Searching for "Magellanic" by Quality Control...
Done.
Searching for "Leave Em Alone (Layton Greene, Lil Baby feat. City Girls, PnB Rock)" by Quality Control...
Done.
Searching for "Ride (Lil Baby feat. Rylo Rodriguez, 24Heavy)" by Quality Control...
Done.
Searching for "Wiggle It (French Montana feat. City Girls)" by Quality Control...
Done.
Searching for "Str

Done.
Searching for "Nothing Revealed / Everything Denied" by The 1975...
Done.
Searching for "Tonight (I Wish I Was Your Boy)" by The 1975...
Done.
Searching for "Shiny Collarbone" by The 1975...
Done.
Searching for "If You’re Too Shy (Let Me Know)" by The 1975...
Done.
Searching for "Playing On My Mind" by The 1975...
Done.
Searching for "Having No Head" by The 1975...
Done.
Searching for "What Should I Say" by The 1975...
Done.
Searching for "Bagsy Not In Net" by The 1975...
Done.
Searching for "Don’t Worry" by The 1975...
Done.
Searching for "Guys" by The 1975...
Done.
Pulling album id 4h3HXlnt6lryGzGbWmcFuY
Searching for "Level of Concern" by Twenty One Pilots...
Done.
Pulling album id 5CEfb4A8SgiCOY6R7c7s34
Searching for "Heartbreaker" by Kaash Paige...
Done.
Searching for "WMT" by Kaash Paige...
Done.
Searching for "64'" by Kaash Paige...
Done.
Searching for "Fake Friends" by Kaash Paige...
Done.
Searching for "Options" by Kaash Paige...
Done.
Searching for "Kaash's Interlude" b

...

Done.
Searching for "10 F*CKS (with Mansa)" by Tory Lanez...
Done.
Searching for "Dope Boy's Diary" by Tory Lanez...
Done.
Searching for "Accidents Happen (with Lil Tjay)" by Tory Lanez...
Done.
Searching for "Broke In A Minute" by Tory Lanez...
Done.
Searching for "P.A.I.N" by Tory Lanez...
Done.
Searching for "Adidas" by Tory Lanez...
Done.
Searching for "Who Needs Love" by Tory Lanez...
Done.
Searching for "Do The Most" by Tory Lanez...
Done.
Searching for "Penthouse Red" by Tory Lanez...
Done.
Searching for "Letter To The City 2" by Tory Lanez...
Done.
Searching for "Back In Business" by Tory Lanez...
Done.
Searching for "D.N.D." by Tory Lanez...
Done.
Searching for "MSG 4 GOD'S CHILDREN" by Tory Lanez...
Done.
Pulling album id 7q6zjWaLw8bN5nYiBWjrus
Searching for "THE SCOTTS" by THE SCOTTS...
Done.
Pulling album id 5Gn3fFzlWL89j0hGumtXb5
Searching for "Ghost" by Noah Cyrus...
Done.
Searching for "I Got So High That I Saw Jesus" by Noah Cyrus...
Done.
Searching for "Liar" by Noah C

Done.
Searching for "The Crying Game" by Nicki Minaj...
Done.
Searching for "Get On Your Knees" by Nicki Minaj...
Done.
Searching for "Feeling Myself" by Nicki Minaj...
Done.
Searching for "Only" by Nicki Minaj...
Done.
Searching for "Want Some More" by Nicki Minaj...
Done.
Searching for "Four Door Aventador" by Nicki Minaj...
Done.
Searching for "Favorite" by Nicki Minaj...
Done.
Searching for "Buy A Heart" by Nicki Minaj...
Done.
Searching for "Trini Dem Girls" by Nicki Minaj...
Done.
Searching for "Anaconda" by Nicki Minaj...
Done.
Searching for "The Night Is Still Young" by Nicki Minaj...
Done.
Searching for "Pills N Potions" by Nicki Minaj...
Done.
Searching for "Bed Of Lies" by Nicki Minaj...
Done.
Searching for "Grand Piano" by Nicki Minaj...
Done.
Searching for "Big Daddy" by Nicki Minaj...
Done.
Searching for "Shanghai" by Nicki Minaj...
Done.
Searching for "Win Again" by Nicki Minaj...
Done.
Pulling album id 4Uv86qWpGTxf7fU7lG5X6F
Searching for "Intro" by Kanye West...
Done.


Done.
Searching for "For The Night (feat. Lil Baby & DaBaby)" by Pop Smoke...
Done.
Searching for "44 BullDog" by Pop Smoke...
Done.
Searching for "Gangstas" by Pop Smoke...
Done.
Searching for "Yea Yea" by Pop Smoke...
Done.
Searching for "Creature (feat. Swae Lee)" by Pop Smoke...
Done.
Searching for "Snitching (feat. Quavo & Future)" by Pop Smoke...
Done.
Searching for "Make It Rain (feat. Rowdy Rebel)" by Pop Smoke...
Done.
Searching for "The Woo (feat. 50 Cent & Roddy Ricch)" by Pop Smoke...
Done.
Searching for "West Coast Shit (feat. Tyga & Quavo)" by Pop Smoke...
Done.
Searching for "Enjoy Yourself (feat. Karol G)" by Pop Smoke...
Done.
Searching for "Mood Swings (feat. Lil Tjay)" by Pop Smoke...
Done.
Searching for "Something Special" by Pop Smoke...
Done.
Searching for "What You Know Bout Love" by Pop Smoke...
Done.
Searching for "Diana (feat. King Combs)" by Pop Smoke...
Done.
Searching for "Got It On Me" by Pop Smoke...
Done.
Searching for "Tunnel Vision (Outro)" by Pop Smok

Specified song does not contain lyrics. Rejecting.
Searching for "ERASE U" by The Kid LAROI...
Done.
Searching for "RUNNING" by The Kid LAROI...
Done.
Searching for "WISH YOU WELL (skit)" by The Kid LAROI...
Specified song does not contain lyrics. Rejecting.
Searching for "NEED YOU MOST (So Sick)" by The Kid LAROI...
Done.
Searching for "SELFISH" by The Kid LAROI...
Done.
Pulling album id 6mJZTV8lCqnwftYZa94bXS
Searching for "Intro : Persona" by BTS...
Done.
Searching for "Boy With Luv (Feat. Halsey)" by BTS...
Done.
Searching for "Make It Right" by BTS...
Done.
Searching for "Jamais Vu" by BTS...
Done.
Searching for "Dionysus" by BTS...
Done.
Searching for "Interlude : Shadow" by BTS...
Done.
Searching for "Black Swan" by BTS...
Done.
Searching for "Filter" by BTS...
Done.
Searching for "My Time" by BTS...
Done.
Searching for "Louder than bombs" by BTS...
Done.
Searching for "ON" by BTS...
Done.
Searching for "UGH!" by BTS...
Done.
Searching for "00:00 (Zero O’Clock)" by BTS...
Done.


KeyboardInterrupt: 

In [15]:
client.close()